In [11]:
import os
import sys
import numpy as np
import pandas as pd
import math


In [12]:
print(os.listdir())
print(os.listdir("../"))

['.ipynb_checkpoints', 'RCN_HeartDataset.ipynb']
['.git', 'Code', 'Data', 'Paper', 'README.md']


In [13]:
os.chdir("../Data/Heart disease UCI")

# Importing the dataset and adding the rules

In [14]:
train_df = pd.read_csv("heart.csv")
train_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [15]:
X = train_df.drop(columns=['target'])
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [16]:
y = train_df[["target"]]
y.head()

,target
0,1
1,1
2,1
3,1
4,1


In [17]:
rules_df = pd.read_csv("rules.csv")
rules_df.head()

,Rule Index,No. Of Instances,Instances,Class1 count,Class0 count,No. Of Terms,Term1 Attribute,Term1 Relation,Term1 Value,Term2 Attribute,...,Term2 Value,Term3 Attribute,Term3 Relation,Term3 Value,Term4 Attribute,Term4 Relation,Term4 Value,Term5 Attribute,Term5 Relation,Term5 Value
0,0,0,NaN,0,0,4,cp,2,0.0,thal,...,2,thal,1,1,sex,-1,0,NaN,-1,-1
1,1,0,NaN,0,0,3,ca,2,0.0,cp,...,0,sex,2,0,NaN,-1,-1,NaN,-1,-1
2,2,0,NaN,0,0,5,slope,2,1.0,thal,...,2,fbs,-1,0,exang,-1,0,age,-1,56
3,3,0,NaN,0,0,4,oldpeak,-1,0.7,fbs,...,0,trestbps,-1,146,thal,-1,2,NaN,-1,-1
4,4,0,NaN,0,0,4,slope,2,1.0,fbs,...,0,age,-1,61,restecg,-1,0,NaN,-1,-1


In [18]:
n_samples=X.shape[0]
n_features=X.shape[1]
print(n_samples,n_features)

303 13


In [19]:
n_samples=X.shape[0]
n_features=X.shape[1]
n_rules=rules_df.shape[0]
print(n_samples,n_features,n_rules)

303 13 13


In [20]:
data_rule=np.zeros((train_df.shape[0],rules_df.shape[0]))
#print(data_rule)
print(data_rule.shape)

(303, 13)


In [21]:
def cond_check(x,j,col,rel,val):
  col=str(col)
  rel=int(rel)
  val=float(val)
  col=col.strip()
  #if(col=='nan'):print(j,col,rel,val)
  #print([(z,type(z)) for z in list(x.columns)])
  #print(col,type(col))
  if(rel==-2):
    return(x[col][j]<val)
  if(rel==-1):
    return(x[col][j]<=val)
  if(rel==0):
    return(x[col][j]==val)
  if(rel==1):
    return(x[col][j]>=val)
  if(rel==2):
    return(x[col][j]>val)
def isNan(s):
  return s!=s
for i in range(0,len(rules_df)):
  NT=int(rules_df["No. Of Terms"][i])
  for j in range(0,len(train_df)):
    if(NT==0):
      data_rule[j][i]=1
      #print("Original icount ="+str(int(rules_df["No. Of Instances"][i])))
      rules_df["No. Of Instances"][i]=str(1+int(rules_df["No. Of Instances"][i]))
      rules_df["Instances"][i]=str(rules_df["Instances"][i])
      if (rules_df["Instances"][i]=='nan'): rules_df["Instances"][i]=str(j)+","
      else: rules_df["Instances"][i]=str(rules_df["Instances"][i])+str(j)+","
      c=int(train_df["target"][j])
      rules_df["Class"+str(c)+" count"][i]=str(1+int(rules_df["Class"+str(c)+" count"][i]))
      continue
    for k in range(1,NT+1):
      check=cond_check(train_df,j,rules_df["Term"+str(k)+" Attribute"][i],rules_df["Term"+str(k)+" Relation"][i],rules_df["Term"+str(k)+" Value"][i])
      #print(i,j,k,check)
      if(check==False):break
      if(check==True and k==NT):
        data_rule[j][i]=1
        #print("Original icount ="+str(int(rules_df["No. Of Instances"][i])))
        rules_df["No. Of Instances"][i]=str(1+int(rules_df["No. Of Instances"][i]))
        rules_df["Instances"][i]=str(rules_df["Instances"][i])
        if (rules_df["Instances"][i]=='nan'): rules_df["Instances"][i]=str(j)+","
        else: rules_df["Instances"][i]=str(rules_df["Instances"][i])+str(j)+","
        c=int(train_df["target"][j])
        rules_df["Class"+str(c)+" count"][i]=str(1+int(rules_df["Class"+str(c)+" count"][i]))
        #print(i,j,k)
        #print(rules_df)
print(rules_df)


/home/ronit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ronit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ronit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ronit/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: 

    Rule Index  No. Of Instances  \
0            0                53   
1            1                63   
2            2                59   
3            3                90   
4            4                40   
5            5                60   
6            6                83   
7            7                31   
8            8                75   
9            9                58   
10          10                55   
11          11               257   
12          12               303   

                                            Instances  Class1 count  \
0   nan2,6,11,14,15,16,17,19,25,28,30,35,36,38,39,...            51   
1   97,139,165,166,168,174,176,179,180,185,186,187...             2   
2   2,3,10,11,12,18,22,27,30,32,40,44,45,46,47,48,...            54   
3   4,5,11,12,16,21,22,27,30,32,33,41,43,44,45,46,...            78   
4   2,32,34,37,40,44,46,47,48,49,50,56,57,58,59,72...            26   
5   23,35,107,119,139,165,166,169,170,174,178,179,...             5  

In [22]:
rules_df.to_csv("rules2.csv",index=False)

In [23]:
for i in range(0,n_rules):
  X["rule"+str(i)]=data_rule[:,i]

In [24]:
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,...,rule3,rule4,rule5,rule6,rule7,rule8,rule9,rule10,rule11,rule12
0,63,1,3,145,233,1,0,150,0,2.3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,37,1,2,130,250,0,1,187,0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,41,0,1,130,204,0,0,172,0,1.4,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,56,1,1,120,236,0,1,178,0,0.8,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,57,0,0,120,354,0,1,163,1,0.6,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


* Now we have the rule satisfaction attributes as a part of the dataset

# Splitting the dataset

In [25]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=42)

In [26]:
print(train_X.shape)
print(test_X.shape)

(227, 26)
(76, 26)


\\
# Setting up the DNN model
\\

In [27]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam

# This returns a tensor
inputs = Input(shape=(n_features+n_rules,))

# a layer instance is callable on a tensor, and returns a tensor
#print(inputs.shape)
#print(inputs[:,0:n_features].shape)
x = Dense(64, activation='relu')(inputs[:,0:n_features])
#print(x.shape)
x = Dense(128, activation='relu',kernel_regularizer='l2')(x)
x=Dropout(0.2)(x)
x = Dense(256, activation='relu',kernel_regularizer='l2')(x)
x=Dropout(0.2)(x)
x = Dense(128, activation='relu',kernel_regularizer='l2')(x)
x=Dropout(0.2)(x)
predictions = Dense(1, activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(train_X, train_y, epochs=1000, validation_split=0.2)

Epoch 1/1000
6/6 [==============================] - 0s 61ms/step - loss: 11.3617 - accuracy: 0.5028 - val_loss: 5.8435 - val_accuracy: 0.4565
Epoch 2/1000
6/6 [==============================] - 0s 12ms/step - loss: 8.1752 - accuracy: 0.4751 - val_loss: 7.1651 - val_accuracy: 0.5435
Epoch 3/1000
6/6 [==============================] - 0s 10ms/step - loss: 6.0920 - accuracy: 0.5138 - val_loss: 4.2801 - val_accuracy: 0.5870
Epoch 4/1000
6/6 [==============================] - 0s 7ms/step - loss: 6.0944 - accuracy: 0.4917 - val_loss: 5.0384 - val_accuracy: 0.5435
Epoch 5/1000
6/6 [==============================] - 0s 7ms/step - loss: 5.2928 - accuracy: 0.5193 - val_loss: 4.2244 - val_accuracy: 0.5652
Epoch 6/1000
6/6 [==============================] - 0s 7ms/step - loss: 4.5307 - accuracy: 0.5746 - val_loss: 3.7982 - val_accuracy: 0.7609
Epoch 7/1000
6/6 [==============================] - 0s 7ms/step - loss: 4.1848 - accuracy: 0.5304 - val_loss: 3.7904 - val_accuracy: 0.5435
Epoch 8/1000
6/6

In [29]:
model.evaluate(train_X,train_y)

8/8 [==============================] - 0s 1ms/step - loss: 0.2871 - accuracy: 0.9207


[0.28712764382362366, 0.9207048416137695]

In [30]:
model.evaluate(test_X,test_y)

3/3 [==============================] - 0s 2ms/step - loss: 0.5965 - accuracy: 0.8158


[0.5964792370796204, 0.8157894611358643]

\\
# Setting up the RCN model
\\

In [31]:
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)              

In [32]:
import tensorflow as tf
import keras
from keras import layers
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs,a,inp,num_features):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        self.dense=keras.layers.Dense(num_outputs,activation=a)
        self.model_inputs=inp
        self.model_num_features=num_features
    

    def call(self, input):
        return tf.multiply(self.dense(input),self.model_inputs[:,self.model_num_features:])

class AvgLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(AvgLayer, self).__init__()
        self.num_outputs = num_outputs


    def call(self, input):
        self.arr=np.ones((input.shape[1],1))
        self.arr=(1.0/input.shape[1])*self.arr
        return tf.matmul(input,self.arr)

In [33]:
"""
model.pop()
model.trainable=False
model.add(MyDenseLayer(n_rules,'softmax',model.inputs,n_features))
model.add(AvgLayer(1))
model.summary()
"""



"\nmodel.pop()\nmodel.trainable=False\nmodel.add(MyDenseLayer(n_rules,'softmax',model.inputs,n_features))\nmodel.add(AvgLayer(1))\nmodel.summary()\n"

In [34]:
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)              

In [35]:
print(model.layers[-2].output.shape)
model.layers.pop()
print(model.summary())
#x=MyDenseLayer(n_rules,'softmax',model.inputs,n_features)(model.layers[-2].output)
#x = Dense(64, activation='relu')(model.layers[-2].output)
#x = Dense(64, activation='relu')(model.layers[-2].output)
#outputs=Dense(1,activation="sigmoid")(x)
#model=Model(model.inputs,outputs)
#print(model.summary())

(None, 128)
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 13)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)  